In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
sc = SparkContext(appName="PythonStreamingQueueStream")

In [3]:
ssc = StreamingContext(sc,5)

In [4]:
import random
from operator import add

In [5]:
def gen(n):
    k = []
    for i in range(n):
        x = random.uniform(-4,4)
        y = random.uniform(-3,5)
        z = random.uniform(-6,4)
        k.append([x,y,z])
    return k

In [6]:
sample_size = 100000

In [7]:
import time
rddQueue = []
for i in range(5):
    rddQueue += [ssc.sparkContext.parallelize(gen(sample_size))]

In [8]:
def inSphere(x,y,z):
    return x**2+(y-1)**2+(z+2)**2<=16

In [9]:
def inCylinder(x,y,z):
    return 0<= z<=4 and x**2 + y**2 <= 4

In [10]:
def f(item):
    x,y,z = item
    a = inSphere(x,y,z)
    b = inCylinder(x,y,z)
    c = a and b
    d = a or b
    return [("a",a),("b",b),("c",c),("d",d)]

In [11]:
inputStream = ssc.queueStream(rddQueue)
mappedStream = inputStream.flatMap(f)
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b)
sphereSum = reducedStream.map(lambda x:(x[0],x[1]*640/sample_size))
sphereSum.pprint()
ssc.start()

-------------------------------------------
Time: 2021-01-30 18:46:50
-------------------------------------------
('b', 50.8928)
('a', 268.7232)
('d', 299.5456)
('c', 20.0704)

-------------------------------------------
Time: 2021-01-30 18:46:55
-------------------------------------------
('b', 50.4192)
('a', 269.6192)
('d', 300.2624)
('c', 19.776)

-------------------------------------------
Time: 2021-01-30 18:47:00
-------------------------------------------
('b', 50.2528)
('a', 268.2176)
('d', 298.7328)
('c', 19.7376)

-------------------------------------------
Time: 2021-01-30 18:47:05
-------------------------------------------
('b', 50.3488)
('a', 267.232)
('d', 297.5744)
('c', 20.0064)

-------------------------------------------
Time: 2021-01-30 18:47:10
-------------------------------------------
('b', 50.0288)
('a', 269.3696)
('d', 299.2192)
('c', 20.1792)

-------------------------------------------
Time: 2021-01-30 18:47:15
-------------------------------------------

--